In [1]:
from __future__ import print_function
import argparse

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import visdom
import time
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler


def load_data(PATH):    
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
        batch_size=test_batch_size, shuffle=True)

    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
        batch_size=test_batch_size, shuffle=True)

    return train_loader, vest_loader, test_loader


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 100, 5, 1)
        self.conv2 = nn.Conv2d(100, 50, 5, 1)
        self.fc1 = nn.Linear(4 * 4 * 50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


def train(model, train_loader, val_loader, optimizer, epochs, log_interval):
    since = time.time()
    model.train()
    if model is None:
        best_model = model.load_state_dict(torch.load('./models/best_model.pt'))
    best_loss = float("inf")
    for epoch in range(1, epochs + 1):
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            # torch.save(the_model, './models/best_model.pt')
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                           100. * batch_idx / len(train_loader), loss.item()))
        valid_loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in valid_loader:
                output = model(data)
                valid_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
                pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
                correct += pred.eq(target.view_as(pred)).sum().item()
                # print(output)

        valid_loss /= len(valid_loader.dataset)

        print('\n Validation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            valid_loss, correct, len(valid_loader.dataset),
            100. * correct / len(valid_loader.dataset)))

        if valid_loss < best_loss:
            best_loss = valid_loss
            best_model = model
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Loss: {:4f}'.format(best_loss))
    torch.save(best_model.state_dict(), './models/best_model.pt')


def test(test_loader):
    model = Net()
    model = model.load_state_dict(torch.load('./models/best_model.pt'))
    model.eval()
    test_loss = 0
    correct = 0
    outputs = []
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            outputs.append(output)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            # print(output)

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    return test_loss, model, outputs

ImportError: No module named 'torch'

In [ ]:
if True:
    train_loader, valid_loader, test_loader = load_data(train_path, test_path)
    train_batch_size = 300
    test_batch_size = 200

    lr = 0.01
    momentum = 0.001

    epochs = 10

    log_interval = int(len(test_loader.dataset) / train_batch_size)

    valid_size = 0.2
    model = Net()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    train(model, train_loader, valid_loader, optimizer, args.epochs, log_interval)

In [3]:
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D
from keras.layers import Dropout
from keras.utils import np_utils
from keras.callbacks import EarlyStopping  

/Users/alisa/anaconda3/envs/my_env_pc/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
Using TensorFlow backend.


In [4]:
from keras.layers.normalization import BatchNormalization
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist

In [22]:
def model_1():
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=num_pixels, kernel_initializer='he_normal'))
    model.add(Dropout(0.5))
    model.add(Conv2D(128, kernel_size=(4, 4), padding='valid', activation='relu', kernel_initializer='he_normal'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
    model.add(Conv2D(128, kernel_size=(4, 4), padding='valid', activation='relu', kernel_initializer='he_normal'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
    model.add(Conv2D(128, kernel_size=(4, 4), padding='valid', activation='relu', kernel_initializer='he_normal'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
    model.add(Conv2D(128, kernel_size=(3, 3), padding='valid', activation='relu', kernel_initializer='he_normal'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
    model.add(Conv2D(128, kernel_size=(3, 3), padding='valid', activation='relu', kernel_initializer='he_normal'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
    model.add(Conv2D(128, kernel_size=(3, 3), padding='valid', activation='relu', kernel_initializer='he_normal'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))    
    #model.add(BatchNormalization())
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  #optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
              optimizer='adam',
              metrics=['accuracy'])
    
    return model

In [23]:
seed = 7
np.random.seed(seed)

(X_train, y_train), (X_test, y_test) = mnist.load_data()
num_pixels = X_train.shape[1] * X_train.shape[2]

In [24]:
X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')

X_train = X_train/255
X_test = X_test/255

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

num_classes = y_test.shape[1]

In [25]:
es = EarlyStopping(monitor='val_acc', mode='max', min_delta=0.001, patience=50)

In [26]:
model = model_1()
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=300, batch_size=100, verbose=2, callbacks=[es])
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

ValueError: Input 0 is incompatible with layer conv2d_19: expected ndim=4, found ndim=2